<a href="https://colab.research.google.com/github/AylinNaebzadeh/Complex-Dynamic-Network/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Directed** graph (each unordered pair of nodes is saved once): roadNet-CA.txt 
California road network

**Nodes**: 1965206

**Edges**: 5533214

In [1]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
G = nx.read_edgelist("roadNet-CA.txt", nodetype=int, create_using=nx.DiGraph(), comments='#')
number_of_nodes = G.number_of_nodes()
number_of_edges = G.number_of_edges()
print(f"The number of nodes in the graph network is: {number_of_nodes}")
print(f"The number of edges in the graph network is: {number_of_edges}")

 The number of nodes in the graph network is: 1965206
The number of edges in the graph network is: 5533214


In [9]:

degrees = G.degree()
degree_values = [v for k, v in degrees]
sum_of_edges = sum(degree_values)
# Calculating the average is just a matter of division by the number of nodes.
average_degree = sum_of_edges / float(number_of_nodes)
print(f"The average degree in the graph network is: {average_degree}")
